# Tutorial 03: weak imposition of Dirichlet BCs by a Lagrange multiplier (nonlinear problem)

In this tutorial we solve the problem

$$\begin{align*}
&\min_{u} \int_\Omega \left\{ (1 + u^2)\ |\nabla u|^2 - u \right\} dx,\\
&\text{s.t. } u = g\text{ on }\Gamma = \partial \Omega
\end{align*}$$
where $\Omega$ is the unit ball in 2D.

The optimality conditions result in the following nonlinear problem

$$\begin{align*}
&\int_\Omega (1+u^2)\ \nabla u \cdot \nabla v dx + \int_\Omega u \ |\nabla u|^2 v dx = \int_\Omega v dx\\
&\text{s.t. } u = g\text{ on }\Gamma = \partial \Omega
\end{align*}$$


We compare the following two cases:
* **strong imposition of Dirichlet BCs**:
the corresponding weak formulation is
$$
\text{find } u \in V_g \text{ s.t. } \int_\Omega (1+u^2)\ \nabla u \cdot \nabla v dx + \int_\Omega u \ |\nabla u|^2 v dx = \int_\Omega v dx, \quad \forall v \in V_0\\
$$
where
$$
V_g = \{v \in H^1(\Omega): v|_\Gamma = g\},\\
V_0 = \{v \in H^1(\Omega): v|_\Gamma = 0\}.\\
$$
* **weak imposition of Dirichlet BCs**: this requires an introduction of a multiplier $\lambda$ which is restricted to $\Gamma$, and solves
$$
\text{find } w, \lambda \in V \times M \text{ s.t. }\\
\begin{cases}
\int_\Omega (1+u^2)\ \nabla u \cdot \nabla v dx + \int_\Omega u \ |\nabla u|^2 v dx + \int_\Gamma \lambda v = \int_\Omega v, & \forall v \in V,\\
\int_\Gamma w \mu = \int_\Gamma g \mu, & \forall \mu \in M
\end{cases}
$$
where
$$
V = H^1(\Omega),\\
M = L^{2}(\Gamma).\\
$$

This example is a prototypical case of problems containing subdomain/boundary restricted variables (the Lagrange multiplier, in this case).

In [ ]:
import numpy as np
from mpi4py import MPI
from petsc4py import PETSc
from ufl import derivative, grad, inner, Measure, replace, TestFunction, TrialFunction
from dolfinx import DirichletBC, Function, FunctionSpace
from dolfinx.fem import locate_dofs_topological
from dolfinx.io import XDMFFile
from dolfinx.plot import create_vtk_topology
from multiphenicsx.fem import (apply_lifting, assemble_matrix, assemble_matrix_block, assemble_scalar,
                               assemble_vector, assemble_vector_block, BlockVecSubVectorWrapper,
                               create_vector, create_vector_block, create_matrix, create_matrix_block,
                               DofMapRestriction, set_bc)
import pyvista

### Mesh

In [ ]:
with XDMFFile(MPI.COMM_WORLD, "data/circle.xdmf", "r") as infile:
    mesh = infile.read_mesh()
    mesh.topology.create_connectivity_all()
    subdomains = infile.read_meshtags(mesh, name="subdomains")
    boundaries = infile.read_meshtags(mesh, name="boundaries")
facets_Gamma = boundaries.indices[boundaries.values == 1]

In [ ]:
# Define associated measures
dx = Measure("dx")(subdomain_data=subdomains)
ds = Measure("ds")(subdomain_data=boundaries)

In [ ]:
def dolfinx_to_pyvista_mesh(mesh):
    num_cells = mesh.topology.index_map(mesh.topology.dim).size_local
    cell_entities = np.arange(num_cells, dtype=np.int32)
    pyvista_cells, cell_types = create_vtk_topology(mesh, mesh.topology.dim, cell_entities)
    grid = pyvista.UnstructuredGrid(pyvista_cells, cell_types, mesh.geometry.x)
    return grid

In [ ]:
def pyvista_mesh_plot(mesh):
    grid = dolfinx_to_pyvista_mesh(mesh)
    plotter = pyvista.PlotterITK()
    plotter.add_mesh(grid)
    plotter.show()

In [ ]:
pyvista_mesh_plot(mesh)

### Weak imposition of Dirichlet BCs

In [ ]:
# Define a function space
V = FunctionSpace(mesh, ("Lagrange", 2))
M = V.clone()

In [ ]:
# Define restrictions
dofs_V = np.arange(0, V.dofmap.index_map.size_local + V.dofmap.index_map.num_ghosts)
dofs_M_Gamma = locate_dofs_topological(M, boundaries.dim, facets_Gamma)
restriction_V = DofMapRestriction(V.dofmap, dofs_V)
restriction_M_Gamma = DofMapRestriction(M.dofmap, dofs_M_Gamma)
restriction = [restriction_V, restriction_M_Gamma]

In [ ]:
# Define trial and test functions, as well as solution
(du, dl) = (TrialFunction(V), TrialFunction(M))
(u, l) = (Function(V), Function(M))
(v, m) = (TestFunction(V), TestFunction(M))

In [ ]:
# Define problem block forms
g = Function(V)
g.interpolate(lambda x: np.sin(3 * x[0] + 1) * np.sin(3 * x[1] + 1))
F = [inner((1 + u**2) * grad(u), grad(v)) * dx + u * v * inner(grad(u), grad(u)) * dx + l * v * ds - v * dx,
     u * m * ds - g * m * ds]
J = [[derivative(F[0], u, du), derivative(F[0], l, dl)],
     [derivative(F[1], u, du), derivative(F[1], l, dl)]]

In [ ]:
# Class for interfacing with the SNES
class NonlinearLagrangeMultplierBlockProblem(object):
    def __init__(self, F, J, solutions, bcs, P=None):
        self._F = F
        self._J = J
        self._obj_vec = create_vector_block(F, restriction)
        self._solutions = solutions
        self._bcs = bcs
        self._P = P

    def update_solutions(self, x):
        x.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)
        with BlockVecSubVectorWrapper(x, [V.dofmap, M.dofmap], restriction) as x_wrapper:
            for x_wrapper_local, sub_solution in zip(x_wrapper, self._solutions):
                with sub_solution.vector.localForm() as sub_solution_local:
                    sub_solution_local[:] = x_wrapper_local

    def obj(self, snes, x):
        self.F(snes, x, self._obj_vec)
        return self._obj_vec.norm()

    def F(self, snes, x, F_vec):
        self.update_solutions(x)
        with F_vec.localForm() as F_vec_local:
            F_vec_local.set(0.0)
        assemble_vector_block(F_vec, self._F, self._J, self._bcs, x0=x,
                              scale=-1.0, restriction=restriction, restriction_x0=restriction)

    def J(self, snes, x, J_mat, P_mat):
        J_mat.zeroEntries()
        assemble_matrix_block(J_mat, self._J, self._bcs, diagonal=1.0,
                              restriction=(restriction, restriction))
        J_mat.assemble()
        if self._P is not None:
            P_mat.zeroEntries()
            assemble_matrix_block(P_mat, self._P, self._bcs, diagonal=1.0,
                                  restriction=(restriction, restriction))
            P_mat.assemble()

In [ ]:
# Create problem
problem = NonlinearLagrangeMultplierBlockProblem(F, J, (u, l), [])
F_vec = create_vector_block(F, restriction=restriction)
J_mat = create_matrix_block(J, restriction=(restriction, restriction))

In [ ]:
# Solve
solution = create_vector_block(F, restriction=restriction)
snes = PETSc.SNES().create(mesh.mpi_comm())
snes.setTolerances(max_it=20)
snes.getKSP().setType("preonly")
snes.getKSP().getPC().setType("lu")
snes.getKSP().getPC().setFactorSolverType("mumps")
snes.setObjective(problem.obj)
snes.setFunction(problem.F, F_vec)
snes.setJacobian(problem.J, J=J_mat, P=None)
snes.setMonitor(lambda _, it, residual: print(it, residual))
snes.solve(None, solution)
problem.update_solutions(solution)  # TODO can this be safely removed?

In [ ]:
def pyvista_scalar_field_plot(mesh, scalar_field, name):
    grid = dolfinx_to_pyvista_mesh(mesh)
    grid.point_arrays[name] = scalar_field.compute_point_values()
    grid.set_active_scalars(name)
    plotter = pyvista.PlotterITK()
    plotter.add_mesh(grid)
    plotter.show()

In [ ]:
pyvista_scalar_field_plot(mesh, u, "u")

In [ ]:
pyvista_scalar_field_plot(mesh, l, "l")

### Strong imposition of Dirichlet BCs

In [ ]:
# Class for interfacing with the SNES
class NonlinearLagrangeMultplierProblem(object):
    def __init__(self, F, J, solution, bcs, P=None):
        self._F = F
        self._J = J
        self._obj_vec = create_vector(F)
        self._solution = solution
        self._bcs = bcs
        self._P = P

    def update_solution(self, x):
        x.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)
        with x.localForm() as _x, self._solution.vector.localForm() as _solution:
            _solution[:] = _x

    def obj(self, snes, x):
        self.F(snes, x, self._obj_vec)
        return self._obj_vec.norm()

    def F(self, snes, x, F_vec):
        self.update_solution(x)
        with F_vec.localForm() as F_vec_local:
            F_vec_local.set(0.0)
        assemble_vector(F_vec, self._F)
        apply_lifting(F_vec, [self._J], [self._bcs], x0=[x], scale=-1.0)
        F_vec.ghostUpdate(addv=PETSc.InsertMode.ADD, mode=PETSc.ScatterMode.REVERSE)
        set_bc(F_vec, self._bcs, x, -1.0)

    def J(self, snes, x, J_mat, P_mat):
        J_mat.zeroEntries()
        assemble_matrix(J_mat, self._J, self._bcs, diagonal=1.0)
        J_mat.assemble()
        if self._P is not None:
            P_mat.zeroEntries()
            assemble_matrix(P_mat, self._P, self._bcs, diagonal=1.0)
            P_mat.assemble()

In [ ]:
# Define problem block forms
u_ex = Function(V)
F_ex = replace(F[0], {u: u_ex, l: 0})
J_ex = derivative(F_ex, u_ex, du)

In [ ]:
# Define Dirichlet BC object on Gamma
dofs_V_Gamma = locate_dofs_topological(V, boundaries.dim, facets_Gamma)
bc_ex = [DirichletBC(g, dofs_V_Gamma)]

In [ ]:
# Create problem
problem_ex = NonlinearLagrangeMultplierProblem(F_ex, J_ex, u_ex, bc_ex)
F_ex_vec = create_vector(F_ex)
J_ex_mat = create_matrix(J_ex)

In [ ]:
# Solve
solution_ex = create_vector(F_ex)
snes = PETSc.SNES().create(mesh.mpi_comm())
snes.setTolerances(max_it=20)
snes.getKSP().setType("preonly")
snes.getKSP().getPC().setType("lu")
snes.getKSP().getPC().setFactorSolverType("mumps")
snes.setObjective(problem_ex.obj)
snes.setFunction(problem_ex.F, F_ex_vec)
snes.setJacobian(problem_ex.J, J=J_ex_mat, P=None)
snes.setMonitor(lambda _, it, residual: print(it, residual))
snes.solve(None, solution_ex)
problem_ex.update_solution(solution_ex)  # TODO can this be safely removed?

In [ ]:
pyvista_scalar_field_plot(mesh, u_ex, "u")

### Comparison and error computation

In [ ]:
u_ex_norm = np.sqrt(mesh.mpi_comm().allreduce(assemble_scalar(inner(grad(u_ex), grad(u_ex)) * dx), op=MPI.SUM))
err_norm = np.sqrt(mesh.mpi_comm().allreduce(assemble_scalar(inner(grad(u_ex - u), grad(u_ex - u)) * dx), op=MPI.SUM))
print("Relative error is equal to", err_norm / u_ex_norm)
assert np.isclose(err_norm / u_ex_norm, 0., atol=1.e-9)